<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
%matplotlib inline
!pip3 install openpifpaf

import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!git clone --branch sda https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git src
!mv src/openpifpaf_animalplugin/ openpifpaf_animalplugin
from src.openpifpaf_animalplugin import AnimalPoseEstimation

fatal: destination path 'src' already exists and is not an empty directory.


In [34]:
!rm -rf data-animalpose/
#!mkdir data-animalpose
!gdown "https://drive.google.com/drive/folders/1xxm6ZjfsDSmv6C9JvbgiGrmHktrUjV5x" --folder
!mv animalpose_keypoint_new/ data-animalpose/
!unzip data-animalpose/images.zip -d data-animalpose/
!rm data-animalpose/images.zip

Retrieving folder list
Processing file 1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF images.zip
Processing file 1bAidtvR3MttetncXz0mfr47vYy7Uf5hm keypoints.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF
To: /content/animalpose_keypoint_new/images.zip
100% 368M/368M [00:01<00:00, 243MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bAidtvR3MttetncXz0mfr47vYy7Uf5hm
To: /content/animalpose_keypoint_new/keypoints.json
100% 2.36M/2.36M [00:00<00:00, 232MB/s]
Download completed
Archive:  data-animalpose/images.zip
   creating: data-animalpose/images/
  inflating: data-animalpose/images/2007_000063.jpg  
  inflating: data-animalpose/images/2007_000175.jpg  
  inflating: data-animalpose/images/2007_000332.jpg  
  inflating: data-animalpose/images/2007_000392.jpg  
  inflating: data-animalpose/images/2007_000464.jpg  
  inflating: data-animalpose/images/2

In [35]:
config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES

{'animal': openpifpaf.plugins.animalpose.animal_kp.AnimalKp,
 'apollo': openpifpaf.plugins.apollocar3d.apollo_kp.ApolloKp,
 'cifar10': openpifpaf.plugins.cifar10.datamodule.Cifar10,
 'cocodet': openpifpaf.plugins.coco.cocodet.CocoDet,
 'cocokp': openpifpaf.plugins.coco.cocokp.CocoKp,
 'crowdpose': openpifpaf.plugins.crowdpose.module.CrowdPose,
 'nuscenes': openpifpaf.plugins.nuscenes.nuscenes.NuScenes,
 'posetrack2018': openpifpaf.plugins.posetrack.posetrack2018.Posetrack2018,
 'posetrack2017': openpifpaf.plugins.posetrack.posetrack2017.Posetrack2017,
 'cocokpst': openpifpaf.plugins.posetrack.cocokpst.CocoKpSt,
 'wholebody': openpifpaf.plugins.wholebody.wholebody.Wholebody,
 'custom_animal': openpifpaf_animalplugin.dataset.AnimalPoseEstimation}

In [36]:
# create a dataset
dataset = AnimalPoseEstimation(image_dir='data-animalpose/images', annotation_file='data-animalpose/keypoints.json')

In [ ]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k16 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=300 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 16 \
    --batch-size 8

INFO:__main__:neural network device: cpu (CUDA available: False, count: 0)
INFO:openpifpaf.network.basenetworks:shufflenetv2k16: stride = 16, output features = 1392
INFO:openpifpaf.network.losses.multi_head:multihead loss: ['custom-animal.cif.c', 'custom-animal.cif.vec', 'custom-animal.cif.scales', 'custom-animal.caf.c', 'custom-animal.caf.vec', 'custom-animal.caf.scales'], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
INFO:openpifpaf.logger:{'type': 'process', 'argv': ['/usr/local/lib/python3.9/dist-packages/openpifpaf/train.py', '--dataset', 'custom_animal', '--basenet=shufflenetv2k16', '--lr=0.00002', '--momentum=0.95', '--epochs=300', '--lr-decay', '160', '260', '--lr-decay-epochs=10', '--weight-decay=1e-5', '--weight-decay=1e-5', '--val-interval', '10', '--loader-workers', '16', '--batch-size', '8'], 'args': {'output': 'outputs/shufflenetv2k16-230422-185501-custom_animal.pkl', 'disable_cuda': False, 'ddp': False, 'local_rank': None, 'sync_batchnorm': True, 'quiet': False, 'debug': False, 'log_st